<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Experiment 1: Train Logistic Regression Classifier

In [4]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment 2: Train Random Forest Classifier

In [5]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.95      0.70      0.81        30

    accuracy                           0.97       300
   macro avg       0.96      0.85      0.89       300
weighted avg       0.97      0.97      0.96       300



### Experiment 3: Train XGBoost

In [6]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [7]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [8]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



<h2 align="center" style="color:blue">Track Experiments Using MLFlow</h2>

In [9]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(C=1, solver='liblinear'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [10]:
reports = []
all_models = []
for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    all_models.append(model)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [11]:
print(reports)

[{'0': {'precision': 0.9454545454545454, 'recall': 0.9629629629629629, 'f1-score': 0.9541284403669725, 'support': 270.0}, '1': {'precision': 0.6, 'recall': 0.5, 'f1-score': 0.5454545454545454, 'support': 30.0}, 'accuracy': 0.9166666666666666, 'macro avg': {'precision': 0.7727272727272727, 'recall': 0.7314814814814814, 'f1-score': 0.749791492910759, 'support': 300.0}, 'weighted avg': {'precision': 0.9109090909090909, 'recall': 0.9166666666666666, 'f1-score': 0.91326105087573, 'support': 300.0}}, {'0': {'precision': 0.9675090252707581, 'recall': 0.9925925925925926, 'f1-score': 0.979890310786106, 'support': 270.0}, '1': {'precision': 0.9130434782608695, 'recall': 0.7, 'f1-score': 0.7924528301886793, 'support': 30.0}, 'accuracy': 0.9633333333333334, 'macro avg': {'precision': 0.9402762517658139, 'recall': 0.8462962962962963, 'f1-score': 0.8861715704873927, 'support': 300.0}, 'weighted avg': {'precision': 0.9620624705697692, 'recall': 0.9633333333333334, 'f1-score': 0.9611465627263633, 'sup

In [12]:
all_models

[LogisticRegression(C=1, solver='liblinear'),
 RandomForestClassifier(max_depth=3, n_estimators=30),
 XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric='logloss',
               feature_types=None, gamma=None, grow_policy=None,
               importance_type=None, interaction_constraints=None,
               learning_rate=None, max_bin=None, max_cat_threshold=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, multi_strategy=None, n_estimators=None,
               n_jobs=None, num_parallel_tree=None, random_state=None, ...),
 XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,


In [13]:
import mlflow

In [17]:
import mlflow

# Connect to the MLflow server
mlflow.set_tracking_uri("http://127.0.0.1:5000")
client = mlflow.tracking.MlflowClient()

# Restore the deleted experiment
experiment_name = "Anomaly Detection"
experiment = client.get_experiment_by_name(experiment_name)

if experiment and experiment.lifecycle_stage == "deleted":
    client.restore_experiment(experiment.experiment_id)
    print(f"Restored experiment: {experiment_name}")


Restored experiment: Anomaly Detection


In [15]:
# Permanently delete the experiment
if experiment and experiment.lifecycle_stage == "deleted":
    client.delete_experiment(experiment.experiment_id)  # Permanent deletion
    print(f"Permanently deleted experiment: {experiment_name}")


Permanently deleted experiment: Anomaly Detection


In [20]:
mlflow.set_experiment('Anomaly Detection')
mlflow.set_tracking_uri('http://127.0.0.1:5000')

for i, element in enumerate(models):
    model_name = element[0]
    model = element[2]
    model = all_models[i] # log the trained model
    report = reports[i]
    params = element[1] 

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('f1-score_macro_avg', report['macro avg']['f1-score'])

        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, 'model')
        else:
            mlflow.sklearn.log_model(model, 'model')

2024/11/24 15:07:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/620432200648113893/runs/e736ef0fc46f427da55696a640667479
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/620432200648113893


2024/11/24 15:07:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/620432200648113893/runs/0b2e881da68f423caeddf3de6f614718
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/620432200648113893


2024/11/24 15:07:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/620432200648113893/runs/2d2bee20e55644fbb7767f1101a4fc94
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/620432200648113893


2024/11/24 15:07:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/620432200648113893/runs/8a6c7d31ae984ddb87a6712900038891
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/620432200648113893


## **Register the Model**

In [26]:
model_name = "XGB-Smote"
run_id = input("Enter Run ID: ")
model_uri = f"runs:/{run_id}/model" # the "model" word comes from the part where we logged the model instance. its name of the artifact 
 
result = mlflow.register_model(
    model_uri, model_name
    # "runs:/d16076a3ec534311817565e6527539c0/sklearn-model", "sk-learn-random-forest-reg"
)


Enter Run ID:  8a6c7d31ae984ddb87a6712900038891


Successfully registered model 'XGB-Smote'.
2024/11/24 15:33:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


## **Load the Registered Model**

In [27]:
model_name = "XGB-Smote"
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"
# if the above format does not work for model_uri the
model = mlflow.xgboost.load_model(model_uri)
y_pred = model.predict(X_test)
y_pred[:5]

array([0, 0, 0, 0, 0])

In [30]:
model_name = "XGB-Smote"
model_version = 1
model_uri = f"models:/{model_name}@challenger"
# if the above format does not work for model_uri the
model = mlflow.xgboost.load_model(model_uri)
y_pred = model.predict(X_test)
y_pred[:5]

array([0, 0, 0, 0, 0])

## **Production Environment**

In [31]:
client = mlflow.MlflowClient()

dev_model_uri = f"models:/{model_name}@challenger"
prod_model = "anomaly-detection-prod"

client.copy_model_version(src_model_uri=dev_model_uri, dst_name=prod_model)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1732443136938, current_stage='None', description='', last_updated_timestamp=1732443136938, name='anomaly-detection-prod', run_id='8a6c7d31ae984ddb87a6712900038891', run_link='', source='models:/XGB-Smote/1', status='READY', status_message='', tags={}, user_id='', version='1'>

## **Load the Production Model**

In [32]:
# model_name = prod_model
model_uri = f"models:/{prod_model}@champion"
# if the above format does not work for model_uri the
model = mlflow.xgboost.load_model(model_uri)
y_pred = model.predict(X_test)
y_pred[:5]

array([0, 0, 0, 0, 0])